# Jim's Machine Learning Testbed



## Jupyter Notes
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

# Basic Python interpreter check

In [1]:
# Does anything work
print("Hello World!")

# An IPython (Jupyter) thing
%env



Hello World!


{'HOMEBREW_PREFIX': '/opt/homebrew',
 'MANPATH': '/opt/homebrew/share/man::',
 'COMMAND_MODE': 'unix2003',
 'INFOPATH': '/opt/homebrew/share/info:',
 'SHELL': '/bin/zsh',
 'PYTHONPATH': '/Users/jcoles/Source/jkc/testbeds/jims-ml-sandbox:/Users/jcoles/Applications/DataSpell.app/Contents/plugins/python-ce/helpers/pydev:/Users/jcoles/Applications/DataSpell.app/Contents/plugins/python-ce/helpers-pro/jupyter_debug',
 '__CFBundleIdentifier': 'com.jetbrains.dataspell',
 'TMPDIR': '/var/folders/xc/32y9xdg50zv0b_kvssbpfc_00000gn/T/',
 'LC_ALL': 'en_US.UTF-8',
 'JBOSS_HOME': '/Users/jcoles/java/wildfly-29.0.0.Final',
 'PKG_CONFIG_PATH': '/Users/jcoles/.opam/default/lib/pkgconfig:',
 'TOOLBOX_VERSION': '2.5.3.37797',
 'DISPLAY': '/private/tmp/com.apple.launchd.M4apHXqJbV/org.macosforge.xquartz:0',
 'HOME': '/Users/jcoles',
 'HOMEBREW_REPOSITORY': '/opt/homebrew',
 'OPAMNOENVNOTICE': 'true',
 'PATH': '/Users/jcoles/Source/jkc/testbeds/jims-ml-sandbox/.venv/bin:/Users/jcoles/Library/pnpm:/Users/jco

Use a core Python package, numpy:

In [2]:
import numpy as np
np.add(1,2)

3

# Machine Learning Engine #2: OpenAI
OpenAI requires a paid API key. Set the key in an os environment variable, e.g.:

`export OPENAI_API_KEY=sk-proj-12fKGWJW ...`

A "hello world", OpenAI API-based _Chat Completion_:

In [4]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            # "content": "Write a haiku about recursion in programming."
            "content": "Hello, OpenAI."
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
